We created a CNN to identify the disease a plant is suffering from. A classification model including different layers, chosing hyperparameters, building, training and testing the model. Tf and Keras are used.

In [ ]:
from google.colab import drive #to load data directly from our drive !! We need dataset for this program so just see the logic and code.

#this will prompt for authorization
drive.mount('/content/drive')

In [ ]:
 #locate the drive file and check if its done
 !ls "/content/drive/My Drive"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
import os
from os import listdir
from PIL import Image
from sklearn.preprocessing import label_binarize, LabelBinarizer
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, array_to_img
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dropout, Dense
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
from keras.utils import to_categorical


In [ ]:
#Plottinng 12 images to check dataset
plt.figure(figsize=(12,12))
path = "/content/drive/My Drive/Plant_image/Potato__Early_blight"
for i in range(1,17):
  plt.subplot(4,4,i)
  plt.tight_layout()
  rand_img = imread(path +'/'+random.choice(sorted(os.listdir(path))))
  plt.imshow(rand_img)
  plt.xlabel(rand_img.shape[1], fontsize = 10) #width of image
  plt.ylabel(rand_img.shape[0], fontsize = 10) #height of image


In [ ]:
#converting image to array coz we will be normalizing our data
def convert_image_to_array(image_dir):
  try:
    image = cv2.imread(image_dir)
    if image is not None:
      image = cv2.resize(image, (256,256))
      #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      return img_to_array(image)
    
    else:
      return np,array([])
  except Exception as e:
    print(f"Error: {e}")
    return None

In [ ]:
dir="/content/drive/My Drive/Plant_image"
root_dir = listdir(dir)
image_list, label_list = [], []
all_labels = ['Corn-Common-rust', 'Potato-Early-Blight', 'Tomato-Bacterial-spot']
binary_labels = [0,1,2]
temp = -1

#reading and converting all images to numpy array
for directoy in root_dir:
  plant_image_list = listdir(f"{dir}/{directory}")
  temp+=1
  for files in plant_image_list:
    image_path = f"{dir}/{directory}/{files}"
    image_list.append(convert_image_to_array(image_path))
    label_list.append(binary_labels[temp])

In [ ]:
#visualize the number of class count, to see if the data is balanced or not
label_counts = pd.DataFrame(label_list).value_counts()
label_counts.head()

In [ ]:
# observe shape of the image and total number of images which is the length of the label list only
image_list[0].shape

label_list = np.array(label_list)
label_list.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_size=10)

In [ ]:
#normalize the dataset of our images. As pixel values ranges from 0 to 255 so we will divide by 255 to normalize them to bw 0 and 1
x_train = np.array(x_train, dtype=np.float16) / 255.0
x_test = np.array(x_test, dtype=np.float16) / 255.0
x_train = x_train.reshape(-1, 256, 256, 3)
x_test = x_test.reshape(-1, 256, 256, 3)

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
cnn = Sequential()
cnn.add(Conv2D(32, (3,3), activation='relu', padding="same", input_shape=(256,256,3)))
cnn.add(MaxPooling(pool_size=(3,3)))
cnn.add(Conv2D(64, (3,3), activation='relu', padding="same")
cnn.add(MaxPooling(pool_size=(2,2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(3, activation='softmax'))
cnn.summary()

#Conv2D creates a convolutional kernel that is convolved with the input layer to produce the output tensor.
#maxpooling is a downsampling technique
#Dense layers gives the output as a dot product of input and the kernel

In [ ]:
cnn.compile(optimizer=Adam(0.0001), loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
#data spliting into training and validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [ ]:
#training the model
r = cnn.fit(x_train, y_train, epochs=50, batch_size=128, validation_data = (x_val, y_val))
#training and validation accuracy tell the performance of our model

In [ ]:
#Saving the model using different techniques
cnn.save("/content/drive/My Drive/plant_disease.h5")

#serialize model to json
json_model = cnn.to_json()
#save model using json file
with open('/content/drive/My Drive/plant_model.json', 'w') as json_file:
  json_file.write(json_model)

#saving the weights if the model
cnn.save_weights("/content/drive/My Drive/plant_model_weights.h5")

In [ ]:
#plot the training history and accuracy of the model for the training history
plt.figure(figsize=(12,5))
plt.plot(r.history['accuracy'], color='r')
plt.plot(r.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'val'])
plt.show()


In [ ]:
#evaluating the accuracy of the model
print("Calculating the accuracy")
scores = cnn.evaluate(x_test, y_test)
print(f"Test Accuracy: {score[1]*100}")

In [ ]:
#now predicting on the test dataset 
y_pred = cnn.predict(x_test)

In [ ]:
#plotting test images just to compare
img = array_to_img(x_test[10])
img

In [ ]:
#Finding argmax or max value from the prediction list and comparing original and predictes
print("Originally: ", all_labels[np.argmax(y_test[10])])
print("Predicted: ", all_labels[np.argmax(y_pred[10])])
